In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy

## Load Abalone
`load_abalone` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [2]:
def load_abalone(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["sex", "length", "diameter", "height", "whole weight", "shucked weight",
                 "viscera weight", "shell weight", "rings"], axis=1, inplace=True)
    df = df.drop(columns = ['sex'])
    df = normalize(df, list(df.columns), 0, 1)
    return df

## Load Breast Cancer
`load_breast_cancer` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset and imputes missing values with the mean of the column.

In [3]:
def load_breast_cancer(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["sample code number", "clump thickness", "uniformity of cell size", "uniformity of cell shape",
                 "marginal adhesion", "single epithelial cell size",
                 "bare nuclei", "bland chromatin", "normal nucleoli", "mitoses", "class"], axis=1, inplace=True)
    df = df.drop(columns = ["sample code number"])
    df = df.replace('?', np.nan)
    df['bare nuclei'] = df['bare nuclei'].astype(float)
    df = impute_mean(df, 'bare nuclei', int)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df = one_hot_encode(df, list(df.columns))
    return df

## Load Car
`load_car` loads the csv file from the provided path and sets the column names.

In [4]:
def load_car(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"], axis=1, inplace=True)
    df = one_hot_encode(df, list(df.columns))
    return df

## Load Forest Fires
`load_forest_fires` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [5]:
def load_forest_fires(path):
    df = pd.read_csv(path)
    df = df.drop(columns=['X', 'Y', 'month', 'day'])
    df = normalize(df, list(df.columns), 0, 1)
    return df

## Load House Votes
`load_house_votes` loads the csv file from the provided path and sets the column names.

In [6]:
def load_house_votes(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["class", "handicapped infants", "water project cost sharing", "adoption of the budget resolution",
                 "physician fee freeze", "el salvador aid", "religious groups in schools", "anti satellite test ban",
                 "aid to nicaraguan contras", "mx missile", "immigration", "synfuels corporation cutback",
                 "education spending", "superfund right to sue", "crime", "duty free exports",
                 "export administration act south africa"], axis=1, inplace=True)
    df = one_hot_encode(df, list(df.columns))
    return df

## Load Machine
`load_machine` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [7]:
def load_machine(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["vendor", "model", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"], axis=1, inplace=True)
    df = df.drop(["vendor","model", "ERP"], axis=1)
    df = normalize(df, ["MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP"], 0, 1)
    return df

## Impute Mean
`impute_mean` updates the missing values in a column of the dataset with the mean of the specified column

In [8]:
def impute_mean(data, column, data_type):
    _data = deepcopy(data)
    mean = _data[column].mean()
    _data[column] = _data[column].fillna(mean)
    if data_type == int:
        _data[column] = _data[column].astype(np.int64)
    return _data

## Encode Ordinal
`encode_ordinal` encodes ordinal values of a feature based on the relationship list provided.

In [9]:
def encode_ordinal(data, column, relationship):
    _data = deepcopy(data)
    for index, r in enumerate(relationship):
        _data[column] = _data[column].replace(r, index)
        print(f"encoding column: {column} - value: {r} as: {index}")
    return _data

## One Hot Encode
`one_hot_encode` encodes features of the desired columns with one hot values.

In [10]:
def one_hot_encode(data, columns):
    _data = deepcopy(data)
    for _column in columns:
        _data2 = pd.get_dummies(_data[_column]).groupby(level=0,axis=1).max().add_prefix(_column + ' - ')
        _data = pd.concat([_data, _data2], axis=1)
        _data = _data.drop([_column], axis=1)
    return _data

## Discretize Feature
`discretize_feature` bins feature values based on the desired bin type. Frequency type tries to ensure the same number of data points are in each bin and bins can vary in width. Width type specifies the bin width so the number on points in each bin can vary

In [11]:
def discretize_feature(data, columns, number_bins, type_bins):
    _data = deepcopy(data)
    for _column in columns:
        if type_bins == "frequency":
            _data2 = pd.qcut(_data[_column], q = number_bins, precision=0, duplicates='drop')
            _data[_column] = _data2

        if type_bins == "width":
            _data2 = pd.cut(_data[_column], number_bins)
            _data[_column] = _data2

    return _data

## Standardize
`standardize` updates values in training data and test data to the z standard using the mean and standard deviation from the training set for both calculations

In [12]:
def standardize(training_data, test_data, columns):
    _data_train = deepcopy(training_data)
    _data_test = deepcopy(test_data)
    for _column in columns:
        mean = _data_train[_column].mean()
        std = _data_train[_column].std()
        z_train = (_data_train[_column] - mean)/std
        z_test = (_data_test[_column] - mean)/std

        _data_train[_column] = z_train
        _data_test[_column] = z_test
    return _data_train, _data_test

In [13]:
def normalize(data, columns, min, max):
    _data = deepcopy(data)
    for c_index, _column in enumerate(columns):
        x_max = _data[_column].max()
        x_min = _data[_column].min()
        _data[_column] = min + ((_data[_column] - x_min)*(max - min)/(x_max - x_min))
    return _data

## Extract Validation Set
`extract_validation_set` extracts a random 20% sample for the validation set. Evenly samples the groupings of the output. Randomly shuffles the remaining 80%.

In [14]:
def extract_validation_set(data, class_column):
    _data = deepcopy(data)
    y = class_column

    stratify_20 = _data.groupby(y, group_keys=False).sample(frac=0.20)
    stratify_80 = _data.drop(stratify_20.index).sample(frac=1)

    return stratify_80, stratify_20

## Create Train Test
`create_train_test` takes in the folds of a k-fold to create a test set from 1 fold and a training set from the remaing 4 folds

In [15]:
def create_train_test(folds, index):
    training = pd.DataFrame()
    test = []
    for i, fold in enumerate(folds):
        if i == index:
            test = fold
        else:
            training = pd.concat([training, fold])
    return training, test

## Stratified K Fold
`stratified_k_fold` takes the provided dataset and creates a specified number of evenly distributed folds of that data while maintaining output grouping distributions

In [16]:
def stratified_k_fold(data, k, class_column):
    y = class_column
    split = []
    unique_keys = data.value_counts(subset=y, normalize=True).keys()
    split_class = [data.loc[data[y] == keys] for keys in unique_keys]

    for class_value in split_class:
        d, m = divmod(len(class_value), k)
        split.append(list(class_value[i * d + min(i, m):(i + 1) * d + min(i + 1, m)] for i in range(k)))

    folds = [pd.concat([split[i][c] for i in range(len(unique_keys))]) for c in range(k)]

    return folds

## K Fold
`k_fold` takes the provided dataset and creates a specified number of evenly distributed folds of that data

In [17]:
def k_fold(data, k):

    d, m = divmod(len(data), k)
    folds = list(data[i * d + min(i, m):(i + 1) * d + min(i + 1, m)] for i in range(k))

    return folds

## K Fold Cross Validation Sets
`k_fold_cross_validation_sets` splits the data into validation and train/test sets. Folds the sets k times. Can fold with stratification or without stratification

In [18]:
def k_fold_cross_validation_sets(data, k, class_column, stratified=True, validation=True):
    if validation:
        train, validation = extract_validation_set(data, class_column)
        if stratified:
            train = stratified_k_fold(train, k, class_column)
            #validation = stratified_k_fold(validation, k, class_column)
        else:
            train = k_fold(train, k)
            #validation = k_fold(validation, k)
        return train, validation
    else:
        if stratified:
            train = stratified_k_fold(data, k, class_column)
        else:
            train = k_fold(data, k)
    return train

## Evaluation
`evaluation` applies an evaluation metric on the predicted values. Able to use a classification score by comparing ground truth to the predicted values to determine how many are correct. Able to use Mean Square Error to determine the distance between the ground truth and the predicted values

In [19]:
def evaluation(ground_truth, predicted_values, metric):
    if metric == 'classification_score':
        count = 0
        for index, value in enumerate(ground_truth):
            if predicted_values[index] == value:
                count += 1
        count = count/len(ground_truth)
        return count
    if metric == 'mse':
        error = sum((np.array(ground_truth) - np.array(predicted_values))**2)/len(ground_truth)
        return error

## Train Linear Classification Network
`train_linear_classification_network` trains a single softmax layer for the output

In [20]:
def train_linear_classification_network(train_features, train_class, eta = 0.2, epochs = 20):
    # initialize weights
    weights = np.random.uniform(-0.01, 0.01, (len(list(train_class.columns)), (len(list(train_features.columns)))))
    # run linear network training
    result = 0
    epoch = 0
    while (result < 1.0) and (epoch < epochs):
        train_features = train_features.sample(frac=1)
        for index, x in train_features.iterrows():
            y = np.array(train_class.loc[index]).ravel()
            # determine output for each class prediction
            activity = np.array(x) * weights
            output_classes = np.exp(activity.sum(axis=1))/np.sum(np.exp(activity.sum(axis=1)))

            # update weights
            for i, value in enumerate(weights):
                weights[i] = weights[i] + eta*(y[i] - output_classes[i]) * x

        # calculate total error
        prediction = predict_linear_network(train_features.sort_index(), weights)
        error = cross_entropy(prediction, np.array(train_class.sort_index()))
        result = classification_result(prediction, np.array(train_class.sort_index()))
        print(f'training epoch {epoch+1}: error: {error} classification score: {result}')
        epoch += 1
    return weights

## Train Linear Regression Network
`train_linear_regression_network` trains a single linear perceptron for the output

In [21]:
def train_linear_regression_network(train_features, train_class, eta = 0.002, epochs = 20):
    # initialize weights
    weights = np.random.uniform(-0.01, 0.01, (len(list(train_class.columns)), (len(list(train_features.columns)))))
    # run linear network training
    epoch = 0
    error = 1
    while (error > 0) and (epoch < epochs):
        train_features = train_features.sample(frac=1)
        for index, x in train_features.iterrows():
            y = np.array(train_class.loc[index]).ravel()
            x = np.array(x)
            # determine output for each class prediction
            activity = np.sum(x * weights)
            weights = weights + eta*(y - activity) * x

        # calculate total error
        prediction = predict_regression_network(train_features.sort_index(), weights)
        error = mean_squared_error(prediction, np.array(train_class.sort_index()))
        print(f'training epoch {epoch+1}: mse: {error}')
        epoch += 1
    return weights

## Initialize Weights
`initialize_weights` set random weights between -0.01 and 0.01 for the network

In [22]:
def initialize_weights(train_features, train_class, hidden_layer_sizes = (10,)):
    weights = []
    activation = []
    for layer, size in enumerate(hidden_layer_sizes):
        # weights for input layer
        if layer == 0:
            weights.append(np.random.uniform(-0.01, 0.01, (size, len(list(train_features.columns)))))
            activation.append(np.zeros(size))
        # weights for hidden layer
        else:
            weights.append(np.random.uniform(-0.01, 0.01, (size, len(weights[layer-1]))))
            activation.append(np.zeros(size))
    # weights for output layer
    weights.append(np.random.uniform(-0.01,0.01, (len(list(train_class.columns)), hidden_layer_sizes[-1])))
    activation.append(np.zeros(len(list(train_class.columns))))
    return weights, activation

## Feed Forward
`feed_forward` feed a sample through the network to get an output

In [23]:
def feed_forward(x, weights, activation):
    for layer, layer_weights in enumerate(weights):
        # calculate input layer activation
        if layer == 0:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * x, axis=1))))
        # calculate softmax layer output
        elif layer == len(weights)-1:
            activation[layer] = np.sum(layer_weights * activation[layer-1], axis=1)
            activation[layer] = np.exp(activation[layer])/np.sum(np.exp(activation[layer]))
        # calculate hidden layer activation
        else:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * activation[layer-1], axis=1))))
    return activation

## Update Weights
`update_weights` update the weights of the network based on the prediction and known output

In [24]:
def update_weights(weights, activation, x, y, eta):
    delta_weights = deepcopy(weights)
    for layer, layer_weights in enumerate(reversed(weights)):
        layer_index = -layer-1
        # output layer update
        if layer_index == -1:
            delta = y - activation[layer_index]
            for index, w in enumerate(weights[layer_index]):
                delta_weights[layer_index][index] = (eta * delta[index] * activation[layer_index-1])
        # input layer update
        elif layer_index == -len(weights):
            delta = np.sum(np.transpose(np.transpose(weights[layer_index+1])*delta), axis=0) * (1 - activation[layer_index])*activation[layer_index]
            for dw_index, dw_layer in enumerate(activation[layer_index]):
                delta_weights[layer_index][dw_index] = eta * delta[dw_index] * x
        # hidden layer update
        else:
            delta = np.sum(np.transpose(np.transpose(weights[layer_index+1])*delta), axis=0) * (1 - activation[layer_index])*activation[layer_index]
            for dw_index, dw_layer in enumerate(activation[layer_index]):
                delta_weights[layer_index][dw_index] = eta * delta[dw_index] * activation[layer_index-1]
    for index, layers in enumerate(weights):
        weights[index] = weights[index] + delta_weights[index]
    return weights

## Get Prediction
`get_prediction` return a list of predictions from a list of samples

In [25]:
def get_prediction(features, weights, activation):
    prediction = []
    for x in features:
        output = feed_forward(x, weights, activation)
        prediction.append(output[-1])
    return prediction

## Train Layered Classification Network
`train_layered_classification_network` train a network of a specified size for classification

In [26]:
def train_layered_classification_network(_train_features, _train_class, hidden_layer_sizes, eta = 0.2, epochs = 20):
    result = 0
    epoch = 0
    weights, activation = initialize_weights(_train_features, _train_class, hidden_layer_sizes)
    train_features = _train_features.to_numpy()
    train_class = _train_class.to_numpy()

    while (result < 1.0) and (epoch < epochs):
        # random sample indexes
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            y = train_class[sample[index]]
            x = train_features[sample[index]]
            # determine output for each class prediction
            activation = feed_forward(x, weights, activation)
            weights = update_weights(weights, activation, x, y, eta)
        # calculate total error
        prediction = get_prediction(train_features, weights, activation)
        error = cross_entropy(prediction, train_class)
        result = classification_result(prediction, train_class)
        print(f'training epoch {epoch+1}: error: {error} classification score: {result}')
        epoch += 1
    return weights, activation

## Feed Forward Regression
`feed_forward_regression` feed a sample through the regression network to get an output

In [27]:
def feed_forward_regression(x, weights, activation):
    for layer, layer_weights in enumerate(weights):
        # calculate input layer activation
        if layer == 0:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * x, axis=1))))
        # calculate softmax layer output
        elif layer == len(weights)-1:
            activation[layer] = np.sum(layer_weights * activation[layer-1], axis=1)
        # calculate hidden layer activation
        else:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * activation[layer-1], axis=1))))
    return activation

## Get Prediction Regression
`get_prediction_regression` return a list of predictions from a list of samples for the regression network

In [28]:
def get_prediction_regression(features, weights, activation):
    prediction = []
    for x in features:
        output = feed_forward_regression(x, weights, activation)
        prediction.append(output[-1])
    return prediction

## Train Layered Regression Network
`train_layered_regression_network` train a network of a specified size for regression

In [29]:
def train_layered_regression_network(_train_features, _train_class, hidden_layer_sizes, eta = 0.2, epochs = 20):
    result = 0
    epoch = 0
    weights, activation = initialize_weights(_train_features, _train_class, hidden_layer_sizes)
    train_features = _train_features.to_numpy()
    train_class = _train_class.to_numpy()

    while (result < 1.0) and (epoch < epochs):
        # random sample indexes
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            y = train_class[sample[index]]
            x = train_features[sample[index]]
            # determine output for each class prediction
            activation = feed_forward_regression(x, weights, activation)
            weights = update_weights(weights, activation, x, y, eta)
        # calculate total error
        prediction = get_prediction_regression(train_features, weights, activation)
        error = mean_squared_error(prediction, train_class)
        print(f'training epoch {epoch+1}: error: {error}')
        epoch += 1
    return weights, activation

## Predict linear network
`predict_linear_network` softmax output for linear network

In [30]:
def predict_linear_network(data, network):
    predict = []
    for index, value in data.iterrows():
        activation = []
        activity = np.dot(network, np.array(value))
        for p in activity:
            activation.append(np.exp(p)/np.sum(np.exp(activity)))
        predict.append(activation)
    return predict

## Predict regression network
`predict_regression_network` regression output for linear network

In [31]:
def predict_regression_network(data, network):
    predict = []
    for index, value in data.iterrows():
        activation = np.dot(network, np.array(value))
        predict.append(activation)
    return predict

## Cross Entropy
`cross_entropy` calculate cross entropy loss

In [32]:
def cross_entropy(prediction, ground_truth):
    error = 0
    for index, truth in enumerate(ground_truth):
        entropy = -1*np.sum(truth*np.log2(prediction[index]))
        error = error + entropy
    return error

## Mean Squared Error
`mean_squared_error` calculate mean squared error loss

In [33]:
def mean_squared_error(prediction, ground_truth):
    mse = 0.5 * np.sum((prediction - ground_truth)**2)
    return mse

## Classification Result
`classification_result` determine if output classified sample correctly

In [34]:
def classification_result(prediction, ground_truth):
    count =0
    for index, value in enumerate(prediction):
        predict = np.argmax(value)
        if ground_truth[index][predict] == 1:
            count += 1
    return count/len(ground_truth)

## Initialize Autoencoder Weights
`initialize_autoencoder_weights` set random weights between -0.01 and 0.01 for the network

In [35]:
def initialize_autoencoder_weights(train_features, hidden_layer_sizes = (10,)):
    weights = []
    activation = []

    weights.append(np.random.uniform(-0.01, 0.01, (hidden_layer_sizes[0], len(list(train_features.columns)))))
    activation.append(np.zeros(hidden_layer_sizes[0]))

    weights.append(np.random.uniform(-0.01,0.01, (len(list(train_features.columns)), hidden_layer_sizes[0])))
    activation.append(np.zeros(len(list(train_features.columns))))
    return weights, activation

In [36]:
def feed_forward_autoencoder(x, weights, activation):
    for layer, layer_weights in enumerate(weights):
        # calculate input layer activation
        if layer == 0:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * x, axis=1))))
        # calculate output layer
        else:
            activation[layer] = 1/(1 + np.exp(-1 * (np.sum(layer_weights * activation[layer-1], axis=1))))
    return activation

In [37]:
def get_prediction_autoencoder(features, weights, activation):
    prediction = []
    for x in features:
        output = feed_forward_autoencoder(x, weights, activation)
        prediction.append(output[-1])
    return prediction

In [38]:
def autoencoder_result(prediction, ground_truth):
    distance = []
    for index, value in enumerate(prediction):
        for i, x_hat in enumerate(value):
            if x_hat > 0.5:
                value[i] = 1
            else:
                value[i] = 0
        distance.append(abs(ground_truth[index] - value))
    distance = np.sum(distance)
    return distance

In [39]:
def autoencoder_result_regression(prediction, ground_truth):
    distance = []
    for index, value in enumerate(prediction):
        distance.append(abs(ground_truth[index] - value))
    distance = np.sum(distance)
    return distance

In [40]:
def train_autoencoder_network(_train_features, _train_class, hidden_layer_sizes, eta = 0.2, epochs = 20):
    auto_result = 1
    result = 0
    epoch = 0
    auto_weights, auto_activation = initialize_autoencoder_weights(_train_features, hidden_layer_sizes)
    weights, activation = initialize_weights(_train_features, _train_class, hidden_layer_sizes)
    train_features = _train_features.to_numpy()
    train_class = _train_class.to_numpy()

    while (auto_result > 0) and (epoch < 200):
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            x = train_features[sample[index]]
            # determine output for each class prediction
            auto_activation = feed_forward_autoencoder(x, auto_weights, auto_activation)
            auto_weights = update_weights(auto_weights, auto_activation, x, x, eta)
        prediction = get_prediction_autoencoder(train_features, auto_weights, auto_activation)
        auto_result = autoencoder_result(prediction, train_features)
        print(f'autoencoder training epoch {epoch+1}: decode layer difference: {auto_result}')
        epoch += 1

    weights[0] = auto_weights[0]
    epoch = 0

    while (result < 1.0) and (epoch < epochs):
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            y = train_class[sample[index]]
            x = train_features[sample[index]]
            # determine output for each class prediction
            activation = feed_forward(x, weights, activation)
            weights = update_weights(weights, activation, x, y, eta)
        # calculate total error
        prediction = get_prediction(train_features, weights, activation)
        error = cross_entropy(prediction, train_class)
        result = classification_result(prediction, train_class)
        print(f'training epoch {epoch+1}: error: {error} classification score: {result}')
        epoch += 1
    return weights, activation

In [41]:
def train_autoencoder_network_regression(_train_features, _train_class, hidden_layer_sizes, eta = 0.2, epochs = 20):
    auto_result = 1
    result = 0
    epoch = 0
    auto_weights, auto_activation = initialize_autoencoder_weights(_train_features, hidden_layer_sizes)
    weights, activation = initialize_weights(_train_features, _train_class, hidden_layer_sizes)
    train_features = _train_features.to_numpy()
    train_class = _train_class.to_numpy()

    while (auto_result > 0) and (epoch < 200):
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            x = train_features[sample[index]]
            # determine output for each class prediction
            auto_activation = feed_forward_autoencoder(x, auto_weights, auto_activation)
            auto_weights = update_weights(auto_weights, auto_activation, x, x, eta)
        prediction = get_prediction_autoencoder(train_features, auto_weights, auto_activation)
        auto_result = autoencoder_result(prediction, train_features)
        print(f'autoencoder training epoch {epoch+1}: decode layer difference: {auto_result}')
        epoch += 1

    weights[0] = auto_weights[0]
    epoch = 0

    while (result < 1.0) and (epoch < epochs):
        sample = np.random.choice(len(train_features), len(train_features), replace=False)
        for index, x in enumerate(train_features):
            y = train_class[sample[index]]
            x = train_features[sample[index]]
            # determine output for each class prediction
            activation = feed_forward_regression(x, weights, activation)
            weights = update_weights(weights, activation, x, y, eta)
        # calculate total error
        prediction = get_prediction_regression(train_features, weights, activation)
        error = mean_squared_error(prediction, train_class)
        print(f'training epoch {epoch+1}: error: {error}')
        epoch += 1
    return weights, activation

In [42]:
def cross_validation(data, class_column, z_norm=False):
    error = []
    result = []
    #creates the validation and test/train sets
    if z_norm:
        folds, validation = k_fold_cross_validation_sets(data, 5, list(data.columns[data.columns.str.contains(class_column)])[0], stratified=False)
    else:
        folds, validation = k_fold_cross_validation_sets(data, 5, list(data.columns[data.columns.str.contains(class_column)])[0])
    #run each fold as the test set
    for index, fold in enumerate(folds):
        #create train test sets from folds
        train, test = create_train_test(folds, index)
        if z_norm:
            x_train = train[train.columns[~train.columns.str.contains(class_column)]]
            y_train = train[train.columns[train.columns.str.contains(class_column)]]
            x_test = test[test.columns[~test.columns.str.contains(class_column)]]
            y_test = test[test.columns[test.columns.str.contains(class_column)]]
            network = train_linear_regression_network(x_train, y_train)
            prediction = predict_regression_network(x_test, network)
            error.append(mean_squared_error(prediction, np.array(y_test)))
            print(f'\nfold {index+1}: mse: {error[index]}\n')
        else:
            x_train = train[train.columns[~train.columns.str.contains(class_column)]]
            y_train = train[train.columns[train.columns.str.contains(class_column)]]
            x_test = test[test.columns[~test.columns.str.contains(class_column)]]
            y_test = test[test.columns[test.columns.str.contains(class_column)]]
            network = train_linear_classification_network(x_train, y_train)
            prediction = predict_linear_network(x_test, network)
            error.append(cross_entropy(prediction, np.array(y_test)))
            result.append(classification_result(prediction, np.array(y_test)))
            print(f'\nfold {index+1}: error: {error[index]} classification score: {result[index]}\n')
    if z_norm:
        average_error = sum(error)/len(error)
        print(f'average error: {average_error}')
    else:
        #determine the average of the performance metric over all folds
        average_class = sum(result)/len(result)
        average_error = sum(error)/len(error)
        print(f'average error: {average_error} average classification score {average_class}')
    return average_error

In [43]:
def cross_validation_layered(data, class_column, classifier, network_size, z_norm=False):
    error = []
    result = []
    #creates the validation and test/train sets
    if z_norm:
        folds, validation = k_fold_cross_validation_sets(data, 5, list(data.columns[data.columns.str.contains(class_column)])[0], stratified=False)
    else:
        folds, validation = k_fold_cross_validation_sets(data, 5, list(data.columns[data.columns.str.contains(class_column)])[0])
    #run each fold as the test set
    for index, fold in enumerate(folds):
        #create train test sets from folds
        train, test = create_train_test(folds, index)
        if z_norm:
            x_train = train[train.columns[~train.columns.str.contains(class_column)]]
            y_train = train[train.columns[train.columns.str.contains(class_column)]]
            x_test = test[test.columns[~test.columns.str.contains(class_column)]]
            y_test = test[test.columns[test.columns.str.contains(class_column)]]
            network, activation = classifier(x_train, y_train, network_size)
            prediction = get_prediction_regression(np.array(x_test), network, activation)
            error.append(mean_squared_error(prediction, np.array(y_test)))
            print(f'\nfold {index+1}: mse: {error[index]}\n')
        else:
            x_train = train[train.columns[~train.columns.str.contains(class_column)]]
            y_train = train[train.columns[train.columns.str.contains(class_column)]]
            x_test = test[test.columns[~test.columns.str.contains(class_column)]]
            y_test = test[test.columns[test.columns.str.contains(class_column)]]
            network, activation = classifier(x_train, y_train, network_size)
            prediction = get_prediction(np.array(x_test), network, activation)
            error.append(cross_entropy(prediction, np.array(y_test)))
            result.append(classification_result(prediction, np.array(y_test)))
            print(f'\nfold {index+1}: error: {error[index]} classification score: {result[index]}\n')

    if z_norm:
        average_error = sum(error)/len(error)
        print(f'average error: {average_error}')
    else:
        #determine the average of the performance metric over all folds
        average_class = sum(result)/len(result)
        average_error = sum(error)/len(error)
        print(f'average error: {average_error} average classification score {average_class}')
    return average_error

In [44]:
cancer_data = load_breast_cancer('datasets/breast-cancer-wisconsin.data')
cancer_data.name = 'Breast Cancer'

In [45]:
cancer_data

,clump thickness - 1,clump thickness - 2,clump thickness - 3,clump thickness - 4,clump thickness - 5,clump thickness - 6,clump thickness - 7,clump thickness - 8,clump thickness - 9,clump thickness - 10,...,mitoses - 2,mitoses - 3,mitoses - 4,mitoses - 5,mitoses - 6,mitoses - 7,mitoses - 8,mitoses - 10,class - 2,class - 4
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
458,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
459,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
460,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
cross_validation_layered(cancer_data, 'class', train_autoencoder_network, (25,90))

autoencoder training epoch 1: decode layer difference: 2070.0
autoencoder training epoch 2: decode layer difference: 1880.0
autoencoder training epoch 3: decode layer difference: 1494.0
autoencoder training epoch 4: decode layer difference: 1217.0
autoencoder training epoch 5: decode layer difference: 1039.0
autoencoder training epoch 6: decode layer difference: 802.0
autoencoder training epoch 7: decode layer difference: 660.0
autoencoder training epoch 8: decode layer difference: 546.0
autoencoder training epoch 9: decode layer difference: 453.0
autoencoder training epoch 10: decode layer difference: 338.0
autoencoder training epoch 11: decode layer difference: 289.0
autoencoder training epoch 12: decode layer difference: 245.0
autoencoder training epoch 13: decode layer difference: 182.0
autoencoder training epoch 14: decode layer difference: 166.0
autoencoder training epoch 15: decode layer difference: 142.0
autoencoder training epoch 16: decode layer difference: 122.0
autoencoder 

24.24270055774476

In [47]:
cross_validation_layered(cancer_data, 'class', train_layered_classification_network, (25,90))

training epoch 1: error: 301.5794858460312 classification score: 0.5152542372881356
training epoch 2: error: 275.13083313374796 classification score: 0.5152542372881356
training epoch 3: error: 237.40695731381157 classification score: 0.5152542372881356
training epoch 4: error: 213.70636734852985 classification score: 0.5152542372881356
training epoch 5: error: 70.87884175133611 classification score: 0.9457627118644067
training epoch 6: error: 40.47113967866148 classification score: 0.9694915254237289
training epoch 7: error: 25.477038180982557 classification score: 0.9796610169491525
training epoch 8: error: 20.448001065544627 classification score: 0.9898305084745763
training epoch 9: error: 15.5417668993827 classification score: 0.9932203389830508
training epoch 10: error: 12.397950319179099 classification score: 0.9932203389830508
training epoch 11: error: 7.694284604463291 classification score: 1.0

fold 1: error: 24.91423909746244 classification score: 0.9459459459459459

training

17.129605104547288

In [48]:
cross_validation(cancer_data, 'class')

training epoch 1: error: 38.37712054509941 classification score: 0.976271186440678
training epoch 2: error: 34.259439460880024 classification score: 0.9796610169491525
training epoch 3: error: 22.539628588861472 classification score: 0.9864406779661017
training epoch 4: error: 20.71132154642081 classification score: 0.9932203389830508
training epoch 5: error: 17.433355031378074 classification score: 0.9932203389830508
training epoch 6: error: 16.65664792919012 classification score: 0.9932203389830508
training epoch 7: error: 15.321319056558318 classification score: 0.9932203389830508
training epoch 8: error: 13.298291909277868 classification score: 0.9966101694915255
training epoch 9: error: 12.207396241441199 classification score: 0.9932203389830508
training epoch 10: error: 11.282688801500086 classification score: 0.9932203389830508
training epoch 11: error: 10.320242925792982 classification score: 0.9966101694915255
training epoch 12: error: 9.592125784449388 classification score: 0

26.38419248658321

In [49]:
car_data = load_car('datasets/car.data')
car_data.name = 'car'

In [50]:
car_data

,buying - high,buying - low,buying - med,buying - vhigh,maint - high,maint - low,maint - med,maint - vhigh,doors - 2,doors - 3,...,lug_boot - big,lug_boot - med,lug_boot - small,safety - high,safety - low,safety - med,class - acc,class - good,class - unacc,class - vgood
0,0,0,0,1,0,0,0,1,1,0,...,0,0,1,0,1,0,0,0,1,0
1,0,0,0,1,0,0,0,1,1,0,...,0,0,1,0,0,1,0,0,1,0
2,0,0,0,1,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0
3,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,0
4,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
1724,0,1,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
1725,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1726,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [51]:
cross_validation_layered(car_data, 'class', train_autoencoder_network, (15, 15))

autoencoder training epoch 1: decode layer difference: 590.0
autoencoder training epoch 2: decode layer difference: 45.0
autoencoder training epoch 3: decode layer difference: 24.0
autoencoder training epoch 4: decode layer difference: 7.0
autoencoder training epoch 5: decode layer difference: 3.0
autoencoder training epoch 6: decode layer difference: 1.0
autoencoder training epoch 7: decode layer difference: 0.0
training epoch 1: error: 830.4109671020148 classification score: 0.702262443438914
training epoch 2: error: 490.4576957449076 classification score: 0.8687782805429864
training epoch 3: error: 403.27329261184883 classification score: 0.8805429864253393
training epoch 4: error: 293.2526822894209 classification score: 0.9122171945701357
training epoch 5: error: 272.4040189895977 classification score: 0.92579185520362
training epoch 6: error: 214.9996122985251 classification score: 0.9375565610859729
training epoch 7: error: 162.97491334640884 classification score: 0.9484162895927

18.1371260444805

In [52]:
cross_validation_layered(car_data, 'class', train_layered_classification_network, (15,15))

training epoch 1: error: 916.1785804892687 classification score: 0.702262443438914
training epoch 2: error: 768.6892766639793 classification score: 0.702262443438914
training epoch 3: error: 717.9231709908964 classification score: 0.702262443438914
training epoch 4: error: 651.7780831074484 classification score: 0.7547511312217194
training epoch 5: error: 491.2555840907267 classification score: 0.8760180995475113
training epoch 6: error: 401.3472454753957 classification score: 0.8850678733031674
training epoch 7: error: 440.40355661055753 classification score: 0.8787330316742081
training epoch 8: error: 392.10001637573004 classification score: 0.9076923076923077
training epoch 9: error: 366.62285063590866 classification score: 0.9031674208144796
training epoch 10: error: 332.0445653142864 classification score: 0.9031674208144796
training epoch 11: error: 323.2937414031431 classification score: 0.9122171945701357
training epoch 12: error: 322.4988886968269 classification score: 0.924886

66.48238178427474

In [53]:
cross_validation(car_data, 'class')

training epoch 1: error: 460.11644001103906 classification score: 0.8714932126696833
training epoch 2: error: 392.1570898507258 classification score: 0.8923076923076924
training epoch 3: error: 381.0680316511004 classification score: 0.902262443438914
training epoch 4: error: 331.7993409788624 classification score: 0.9104072398190045
training epoch 5: error: 328.0852456557689 classification score: 0.902262443438914
training epoch 6: error: 321.3073456468773 classification score: 0.9239819004524887
training epoch 7: error: 294.66456477378307 classification score: 0.9266968325791856
training epoch 8: error: 314.3482558390268 classification score: 0.9004524886877828
training epoch 9: error: 283.70960474425203 classification score: 0.9194570135746606
training epoch 10: error: 283.2443598741395 classification score: 0.9239819004524887
training epoch 11: error: 318.9353835923742 classification score: 0.9158371040723982
training epoch 12: error: 306.311635366196 classification score: 0.923076

79.48101272801034

In [79]:
house_votes_data = load_house_votes('datasets/house-votes-84.data')
house_votes_data.name = 'house votes'

In [80]:
house_votes_data

,class - democrat,class - republican,handicapped infants - ?,handicapped infants - n,handicapped infants - y,water project cost sharing - ?,water project cost sharing - n,water project cost sharing - y,adoption of the budget resolution - ?,adoption of the budget resolution - n,...,superfund right to sue - y,crime - ?,crime - n,crime - y,duty free exports - ?,duty free exports - n,duty free exports - y,export administration act south africa - ?,export administration act south africa - n,export administration act south africa - y
0,0,1,0,1,0,0,0,1,0,1,...,1,0,0,1,0,1,0,0,0,1
1,0,1,0,1,0,0,0,1,0,1,...,1,0,0,1,0,1,0,1,0,0
2,1,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,1,0,0,1,0
3,1,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,1,0,0,0,1
4,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,0,1,0,1,0,0,1,0,0,0,...,1,0,0,1,0,1,0,0,0,1
431,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1
432,0,1,0,1,0,1,0,0,0,1,...,1,0,0,1,0,1,0,0,0,1
433,0,1,0,1,0,0,1,0,0,1,...,1,0,0,1,0,1,0,0,0,1


In [81]:
cross_validation_layered(house_votes_data, 'class', train_autoencoder_network, (25, 60))

autoencoder training epoch 1: decode layer difference: 1686.0
autoencoder training epoch 2: decode layer difference: 980.0
autoencoder training epoch 3: decode layer difference: 718.0
autoencoder training epoch 4: decode layer difference: 472.0
autoencoder training epoch 5: decode layer difference: 333.0
autoencoder training epoch 6: decode layer difference: 241.0
autoencoder training epoch 7: decode layer difference: 194.0
autoencoder training epoch 8: decode layer difference: 173.0
autoencoder training epoch 9: decode layer difference: 156.0
autoencoder training epoch 10: decode layer difference: 118.0
autoencoder training epoch 11: decode layer difference: 94.0
autoencoder training epoch 12: decode layer difference: 72.0
autoencoder training epoch 13: decode layer difference: 56.0
autoencoder training epoch 14: decode layer difference: 46.0
autoencoder training epoch 15: decode layer difference: 42.0
autoencoder training epoch 16: decode layer difference: 32.0
autoencoder training e

18.21133508459686

In [82]:
cross_validation_layered(house_votes_data, 'class', train_layered_classification_network, (25, 60))

training epoch 1: error: 242.29412467476527 classification score: 0.6151079136690647
training epoch 2: error: 177.6087446274163 classification score: 0.6151079136690647
training epoch 3: error: 159.01781518071243 classification score: 0.6151079136690647
training epoch 4: error: 150.8304758272887 classification score: 0.6151079136690647
training epoch 5: error: 145.52910040661266 classification score: 0.6151079136690647
training epoch 6: error: 140.85841251551966 classification score: 0.6151079136690647
training epoch 7: error: 137.9660638120817 classification score: 0.6151079136690647
training epoch 8: error: 134.23888181729305 classification score: 0.6151079136690647
training epoch 9: error: 118.61940203618533 classification score: 0.9496402877697842
training epoch 10: error: 41.16177294659475 classification score: 0.9712230215827338
training epoch 11: error: 36.91954809374689 classification score: 0.9676258992805755
training epoch 12: error: 27.41383266763042 classification score: 0.

15.310687985585469

In [83]:
cross_validation(house_votes_data, 'class')

training epoch 1: error: 28.60425237116041 classification score: 0.9712230215827338
training epoch 2: error: 33.21168037542073 classification score: 0.9748201438848921
training epoch 3: error: 19.435785753014017 classification score: 0.9856115107913669
training epoch 4: error: 20.714818130797223 classification score: 0.9784172661870504
training epoch 5: error: 11.671033561285565 classification score: 0.9892086330935251
training epoch 6: error: 11.58465704792062 classification score: 0.9892086330935251
training epoch 7: error: 10.489830216806322 classification score: 0.9892086330935251
training epoch 8: error: 16.600191295145933 classification score: 0.9820143884892086
training epoch 9: error: 21.008787529845545 classification score: 0.9820143884892086
training epoch 10: error: 7.858500891861051 classification score: 0.9928057553956835
training epoch 11: error: 14.265614253521983 classification score: 0.9856115107913669
training epoch 12: error: 8.071801564542842 classification score: 0

15.877644590619212

In [59]:
machine_data = load_machine('datasets/machine.data')
machine_data.name = 'machine'

In [60]:
machine_data

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
0,0.072825,0.006012,0.092843,1.000,0.307692,0.727273,0.167832
1,0.008092,0.248497,0.499499,0.125,0.153846,0.181818,0.229895
2,0.008092,0.248497,0.499499,0.125,0.153846,0.181818,0.187063
3,0.008092,0.248497,0.499499,0.125,0.153846,0.181818,0.145105
4,0.008092,0.248497,0.249249,0.125,0.153846,0.090909,0.110140
...,...,...,...,...,...,...,...
204,0.072151,0.029309,0.124124,0.000,0.019231,0.045455,0.031469
205,0.054619,0.029309,0.124124,0.125,0.038462,0.045455,0.034965
206,0.072825,0.060621,0.124124,0.000,0.038462,0.079545,0.040210
207,0.312205,0.014028,0.124124,0.125,0.000000,0.000000,0.053322


In [76]:
cross_validation_layered(machine_data, 'PRP', train_autoencoder_network_regression, (5,10), z_norm=True)

autoencoder training epoch 1: decode layer difference: 109.66912667871297
autoencoder training epoch 2: decode layer difference: 109.66912667871297
autoencoder training epoch 3: decode layer difference: 109.66912667871297
autoencoder training epoch 4: decode layer difference: 109.66912667871297
autoencoder training epoch 5: decode layer difference: 109.66912667871297
autoencoder training epoch 6: decode layer difference: 109.66912667871297
autoencoder training epoch 7: decode layer difference: 109.66912667871297
autoencoder training epoch 8: decode layer difference: 109.66912667871297
autoencoder training epoch 9: decode layer difference: 109.66912667871297
autoencoder training epoch 10: decode layer difference: 109.66912667871297
autoencoder training epoch 11: decode layer difference: 109.66912667871297
autoencoder training epoch 12: decode layer difference: 109.66912667871297
autoencoder training epoch 13: decode layer difference: 109.66912667871297
autoencoder training epoch 14: dec

0.32828978003769055

In [77]:
cross_validation_layered(machine_data, 'PRP', train_layered_regression_network, (5,10), z_norm=True)

training epoch 1: error: 1.714037596688974
training epoch 2: error: 1.7576812420134116
training epoch 3: error: 1.8215859370876284
training epoch 4: error: 1.9837295244263453
training epoch 5: error: 3.474980996136825
training epoch 6: error: 2.4502974017584203
training epoch 7: error: 1.7712054051085566
training epoch 8: error: 1.7463779785680777
training epoch 9: error: 1.6688157123452234
training epoch 10: error: 1.8123432318520887
training epoch 11: error: 1.527382969579361
training epoch 12: error: 1.6419640479766455
training epoch 13: error: 1.6571342144804482
training epoch 14: error: 1.516998452378821
training epoch 15: error: 1.656411732827283
training epoch 16: error: 1.542024524997142
training epoch 17: error: 1.6907106780982368
training epoch 18: error: 1.6667437454797058
training epoch 19: error: 1.5090140718462923
training epoch 20: error: 1.5866090834600075

fold 1: mse: 0.48635591727493066

training epoch 1: error: 1.9034274263923323
training epoch 2: error: 1.676506288

0.641826521268744

In [78]:
cross_validation(machine_data, 'PRP', z_norm=True)

training epoch 1: mse: 2.373676146223765
training epoch 2: mse: 2.173380401770946
training epoch 3: mse: 1.9929555702321546
training epoch 4: mse: 1.8310052883117551
training epoch 5: mse: 1.6852107185349587
training epoch 6: mse: 1.5536622903118562
training epoch 7: mse: 1.435610134808471
training epoch 8: mse: 1.3292847097706997
training epoch 9: mse: 1.233478901963772
training epoch 10: mse: 1.1474386697776158
training epoch 11: mse: 1.0696218812074374
training epoch 12: mse: 0.9997363241938064
training epoch 13: mse: 0.9369413766419477
training epoch 14: mse: 0.880428880708636
training epoch 15: mse: 0.8292010613195842
training epoch 16: mse: 0.7836177088961525
training epoch 17: mse: 0.7423278890576037
training epoch 18: mse: 0.704929985455976
training epoch 19: mse: 0.6714020173123809
training epoch 20: mse: 0.6409792637920231

fold 1: mse: 0.052045326329703524

training epoch 1: mse: 2.4021994047345396
training epoch 2: mse: 2.2178377074292976
training epoch 3: mse: 2.0504311912

0.18641334878512264

In [64]:
forest_data = load_forest_fires('datasets/forestfires.data')
forest_data.name = 'forest fires'

In [65]:
forest_data

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,0.870968,0.086492,0.101325,0.090909,0.192926,0.423529,0.700000,0.00000,0.000000
1,0.927742,0.118194,0.775419,0.119430,0.508039,0.211765,0.055556,0.00000,0.000000
2,0.927742,0.146795,0.796294,0.119430,0.398714,0.211765,0.100000,0.00000,0.000000
3,0.941935,0.110958,0.081623,0.160428,0.196141,0.964706,0.400000,0.03125,0.000000
4,0.910968,0.172984,0.110590,0.171123,0.295820,0.988235,0.155556,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...
512,0.811613,0.191592,0.771315,0.033868,0.823151,0.200000,0.255556,0.00000,0.005904
513,0.811613,0.191592,0.771315,0.033868,0.633441,0.658824,0.600000,0.00000,0.049769
514,0.811613,0.191592,0.771315,0.033868,0.610932,0.647059,0.700000,0.00000,0.010231
515,0.976774,0.499311,0.711622,0.201426,0.752412,0.317647,0.400000,0.00000,0.000000


In [74]:
cross_validation_layered(forest_data, 'area', train_autoencoder_network_regression, (5,10), z_norm=True)

autoencoder training epoch 1: decode layer difference: 818.909644207064
autoencoder training epoch 2: decode layer difference: 818.909644207064
autoencoder training epoch 3: decode layer difference: 818.909644207064
autoencoder training epoch 4: decode layer difference: 716.456156604484
autoencoder training epoch 5: decode layer difference: 704.1571392045396
autoencoder training epoch 6: decode layer difference: 700.0435062251194
autoencoder training epoch 7: decode layer difference: 689.1460730313595
autoencoder training epoch 8: decode layer difference: 668.8834138501643
autoencoder training epoch 9: decode layer difference: 663.5668066701393
autoencoder training epoch 10: decode layer difference: 658.7843057500233
autoencoder training epoch 11: decode layer difference: 648.6072978898484
autoencoder training epoch 12: decode layer difference: 639.9898785951032
autoencoder training epoch 13: decode layer difference: 636.3074554644556
autoencoder training epoch 14: decode layer differe

0.20477270032033928

In [75]:
cross_validation_layered(forest_data, 'area', train_layered_regression_network, (5,10), z_norm=True)

training epoch 1: error: 0.8534199172288401
training epoch 2: error: 0.8527924380117325
training epoch 3: error: 0.928470821851764
training epoch 4: error: 0.8668839306535688
training epoch 5: error: 0.8725467634748891
training epoch 6: error: 0.84297353732261
training epoch 7: error: 0.8531986536340532
training epoch 8: error: 0.8625269965072716
training epoch 9: error: 0.8678732646486426
training epoch 10: error: 0.8403720019470965
training epoch 11: error: 0.8602323915796194
training epoch 12: error: 0.8542272847585438
training epoch 13: error: 2.0502373069451174
training epoch 14: error: 0.864064703570891
training epoch 15: error: 1.06187787294882
training epoch 16: error: 0.8538596926606393
training epoch 17: error: 0.8574208957152725
training epoch 18: error: 0.8624966007854684
training epoch 19: error: 0.8617646967948996
training epoch 20: error: 0.862219418231985

fold 1: mse: 0.03810064661472318

training epoch 1: error: 0.860829433343635
training epoch 2: error: 0.84561579455

0.3745448804696352

In [68]:
cross_validation(forest_data, 'area', z_norm=True)

training epoch 1: mse: 0.6067519760297929
training epoch 2: mse: 0.6061619687431403
training epoch 3: mse: 0.6057963341191944
training epoch 4: mse: 0.6059462251790279
training epoch 5: mse: 0.6051603362432595
training epoch 6: mse: 0.6057749473343559
training epoch 7: mse: 0.6047789015043378
training epoch 8: mse: 0.60474452041555
training epoch 9: mse: 0.6044414035889675
training epoch 10: mse: 0.6046237141800156
training epoch 11: mse: 0.6041629455178965
training epoch 12: mse: 0.6040759754095778
training epoch 13: mse: 0.6039000970550126
training epoch 14: mse: 0.6038535602117401
training epoch 15: mse: 0.6036234657919701
training epoch 16: mse: 0.6035978907003523
training epoch 17: mse: 0.6037123579239632
training epoch 18: mse: 0.6038602323699542
training epoch 19: mse: 0.6032254481710926
training epoch 20: mse: 0.6031252542367739

fold 1: mse: 0.2656800949744482

training epoch 1: mse: 0.37019357635423605
training epoch 2: mse: 0.3665911563781711
training epoch 3: mse: 0.3662582

0.17427294726882497

In [69]:
abalone_data = load_abalone('datasets/abalone.data')
abalone_data.name = 'abalone'

In [70]:
abalone_data

,length,diameter,height,whole weight,shucked weight,viscera weight,shell weight,rings
0,0.513514,0.521008,0.084071,0.181335,0.150303,0.132324,0.147982,0.500000
1,0.371622,0.352941,0.079646,0.079157,0.066241,0.063199,0.068261,0.214286
2,0.614865,0.613445,0.119469,0.239065,0.171822,0.185648,0.207773,0.285714
3,0.493243,0.521008,0.110619,0.182044,0.144250,0.149440,0.152965,0.321429
4,0.344595,0.336134,0.070796,0.071897,0.059516,0.051350,0.053313,0.214286
...,...,...,...,...,...,...,...,...
4172,0.662162,0.663866,0.146018,0.313441,0.248151,0.314022,0.246637,0.357143
4173,0.695946,0.647059,0.119469,0.341420,0.294553,0.281764,0.258097,0.321429
4174,0.709459,0.705882,0.181416,0.415796,0.352724,0.377880,0.305431,0.285714
4175,0.743243,0.722689,0.132743,0.386931,0.356422,0.342989,0.293473,0.321429


In [71]:
cross_validation_layered(abalone_data, 'rings', train_autoencoder_network_regression, (3,10), z_norm=True)

autoencoder training epoch 1: decode layer difference: 4808.8832926314835
autoencoder training epoch 2: decode layer difference: 4742.996972570839
autoencoder training epoch 3: decode layer difference: 4729.889415308417
autoencoder training epoch 4: decode layer difference: 4724.039155798744
autoencoder training epoch 5: decode layer difference: 4723.548158473917
autoencoder training epoch 6: decode layer difference: 4724.864898043699
autoencoder training epoch 7: decode layer difference: 4722.276007586149
autoencoder training epoch 8: decode layer difference: 4721.311069093171
autoencoder training epoch 9: decode layer difference: 4722.62769993209
autoencoder training epoch 10: decode layer difference: 4722.184894799037
autoencoder training epoch 11: decode layer difference: 4721.8491112340225
autoencoder training epoch 12: decode layer difference: 4721.569515623075
autoencoder training epoch 13: decode layer difference: 4731.914893376698
autoencoder training epoch 14: decode layer di

2.2859422966071152

In [72]:
cross_validation_layered(abalone_data, 'rings', train_layered_regression_network, (3,10), z_norm=True)

training epoch 1: error: 30.925654991929022
training epoch 2: error: 28.39111952342882
training epoch 3: error: 24.745586801369363
training epoch 4: error: 19.017018541981095
training epoch 5: error: 17.884583906430763
training epoch 6: error: 19.149815018971548
training epoch 7: error: 17.40632464860544
training epoch 8: error: 17.651676282093717
training epoch 9: error: 18.572801077408513
training epoch 10: error: 20.191711618079637
training epoch 11: error: 13.783250421814412
training epoch 12: error: 12.673538903019335
training epoch 13: error: 12.399552227888833
training epoch 14: error: 14.55072195555346
training epoch 15: error: 15.00672828346552
training epoch 16: error: 15.571287532050519
training epoch 17: error: 17.401361207282875
training epoch 18: error: 12.17453207397839
training epoch 19: error: 13.828229577905283
training epoch 20: error: 11.752591319927078

fold 1: mse: 2.793017087838275

training epoch 1: error: 17.751534139340187
training epoch 2: error: 22.372314595

2.8572987724980865

In [73]:
cross_validation(abalone_data, 'rings', z_norm=True)

training epoch 1: mse: 14.274319256447988
training epoch 2: mse: 13.747673605408487
training epoch 3: mse: 13.310641538842933
training epoch 4: mse: 12.968417209175888
training epoch 5: mse: 12.64759973813668
training epoch 6: mse: 12.395209883875065
training epoch 7: mse: 12.17673678776882
training epoch 8: mse: 12.005305554003735
training epoch 9: mse: 11.843161088425695
training epoch 10: mse: 11.695806472415315
training epoch 11: mse: 11.609807238563386
training epoch 12: mse: 11.467861249523109
training epoch 13: mse: 11.37160389198279
training epoch 14: mse: 11.281586264173786
training epoch 15: mse: 11.206241396338921
training epoch 16: mse: 11.127143597532903
training epoch 17: mse: 11.085457877809043
training epoch 18: mse: 11.028193815013918
training epoch 19: mse: 10.934512551253786
training epoch 20: mse: 10.889430212365294

fold 1: mse: 2.8772961039332223

training epoch 1: mse: 14.431781304702305
training epoch 2: mse: 13.92033418645236
training epoch 3: mse: 13.466501582

2.7506780045682477